In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential # init NN
from keras.layers import Dense, Dropout # create layers in ANN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Getting Dataset

In [102]:
dataset = pd.read_csv('https://raw.githubusercontent.com/lanceeeaton/ANN_Churn_Modeling/master/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Checking for missing values

In [103]:
dataset.isnull().values.any()

False

## Defining independent variables X and dependent variable y

In [0]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,-1].values

## Looking at X

In [105]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


## Looking at y

In [106]:
y[:5]

array([1, 0, 1, 0, 0])

## Encoding Categorical Vars, Making Dummy Vars and avoiding Dummy var Trap

In [0]:
X = pd.get_dummies(X, columns = ['Geography','Gender'])
X = X.drop(columns=['Geography_Spain', 'Gender_Male'])

## Looking at X again

In [108]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Gender_Female
0,619,42,2,0.00,1,1,1,101348.88,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1
3,699,39,1,0.00,2,0,0,93826.63,1,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


##Converting to float to avoid warning

In [0]:
X = X.astype(float)

## Spliting

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [0]:
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

## Building our ANN

In [0]:
def build_classifier(optimizer = 'adam', units = 6):
  """Builds ANN classifier with 11 input neurons, 2 hidden layers each with 7 neurons (rectifier function) and 1 output neuron (sigmoid function)"""
  classifier = Sequential()
  classifier.add(Dense(activation ='relu', input_dim = 11, units = units, kernel_initializer ='uniform')) # units = 2/3(num input nodes + num output nodes)
  classifier.add(Dropout(0.2))
  classifier.add(Dense(activation ='sigmoid', units = 1, kernel_initializer ='uniform')) # out layer
  classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics =['accuracy']) # telling it to use stochastic gradient descent, and cross-entropy loss for a binary output
  return classifier

classifier = KerasClassifier(build_fn = build_classifier())

In [0]:
batch_sizes = [32,64,128,256,512]
epochs = [50,100,500]
optimizers = ['rmsprop', 'adam']
units = [6,7]
grid_params = dict(optimizer = optimizers, epochs = epochs, batch_size = batch_sizes, units = units)

In [0]:
grid_search = GridSearchCV(estimator = classifier, param_grid = grid_params, scoring = 'accuracy', cv = 10)
grid_search = grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_params)
print(best_accuracy)

## Getting our  accuracy scores, for 10 Fold Cross Validation

In [36]:
scores = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
scores

array([0.8425 , 0.83625, 0.83   , 0.82875, 0.8475 , 0.8475 , 0.84   ,
       0.8225 , 0.80875, 0.8675 ])

## The average accuracy score

In [37]:
scores.mean()

0.837125

## Our percent variance

In [38]:
scores.std() * 100

1.521769118493343

## Training our classifier, and predicting on test set

In [0]:
classifier.fit(X_train, y_train)
y_pred_prob = classifier.predict(X_test)
y_pred = (y_pred_prob > 0.5)

## Making a confusion matrix to better understand results

In [40]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1548,   47],
       [ 265,  140]])

## Seeing our actual accuracy

In [41]:
accuracy = (cm[0][0] + cm[1][1])/ len(y_pred)
accuracy

0.844

## Defining a new customer

In [0]:
new_customer = [[600,40, 3, 60000, 2, 1, 1, 50000,1,0,0]]

## Scaling the customer's data

In [0]:
new_customer = standard_scaler.transform(new_customer)

## Making prediction on our new customer

In [44]:
new_customer_pred_prob = classifier.predict(new_customer)
new_customer_pred = (new_customer_pred_prob > 0.5)
new_customer_pred

array([[False]])

This is an exercise that correponds with the Deep Learning A-Z course on Udemy.